# Nature Conservancy -  Fish Detection

### Imports & environment

In [1]:
import os
import random

import numpy as np
import matplotlib.pyplot as plt

from glob import iglob, glob
from models import Vgg16BN

from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img, img_to_array

from scipy.misc import imresize, toimage

#os.getcwd()
ROOT_DIR = '/home/ipl/Data/CAROTID/'
DATA_HOME_DIR = ROOT_DIR + '/data/'
%matplotlib inline

Using TensorFlow backend.


### Config & Hyperparameters

In [2]:
# paths
data_path = DATA_HOME_DIR #+ '/fish_detection'
split_train_path = data_path + '/train/'
valid_path = data_path + '/valid/'
saved_model_path = ROOT_DIR + '/models/'

# data
batch_size = 8
nb_split_train_samples = 1457#3327
nb_full_train_samples = 3
nb_valid_samples = 400
nb_test_samples = 1000
classes = ["NoF", "FISH"]
nb_classes = len(classes)

# model
nb_runs = 1
nb_epoch = 35
aug = True
dropout = 0.3
lr=0.001
archs = ["vggbn"]

### Train the Fish Detector Model

In [3]:
def train():
    vgg = Vgg16BN(size=(227, 227), n_classes=nb_classes, lr=lr,
              batch_size=batch_size, dropout=dropout)

    vgg.build()
    model_fn = saved_model_path + '{val_loss:.2f}-loss_{epoch}epoch' +'_227x227_{0}-dropout_{1}-lr_vggbn.h5'.format(dropout, lr)
    ckpt = ModelCheckpoint(filepath=model_fn, monitor='val_loss',
                           save_best_only=True, save_weights_only=True)

    vgg.fit_val(split_train_path, valid_path, nb_trn_samples=nb_split_train_samples, 
                         nb_val_samples=nb_valid_samples, nb_epoch=nb_epoch, callbacks=[ckpt], aug=True)

    model_path = max(iglob(saved_model_path + '*.h5'), key=os.path.getctime)
    return model_path

model_path = train()

Found 1457 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/35
43s - loss: 2.9005 - acc: 0.5841 - val_loss: 4.0163 - val_acc: 0.5275
Epoch 2/35
41s - loss: 1.2167 - acc: 0.6452 - val_loss: 4.3810 - val_acc: 0.4875
Epoch 3/35
41s - loss: 0.9737 - acc: 0.6712 - val_loss: 3.4848 - val_acc: 0.4900
Epoch 4/35
41s - loss: 0.7360 - acc: 0.7310 - val_loss: 1.9202 - val_acc: 0.5025
Epoch 5/35
41s - loss: 0.7922 - acc: 0.7165 - val_loss: 1.4792 - val_acc: 0.5200
Epoch 6/35
41s - loss: 0.6107 - acc: 0.7680 - val_loss: 2.1424 - val_acc: 0.5150
Epoch 7/35
40s - loss: 0.6642 - acc: 0.7563 - val_loss: 1.8185 - val_acc: 0.5150
Epoch 8/35
41s - loss: 0.6377 - acc: 0.7563 - val_loss: 1.7128 - val_acc: 0.4975
Epoch 9/35
41s - loss: 0.5192 - acc: 0.8106 - val_loss: 1.7927 - val_acc: 0.4875
Epoch 10/35
41s - loss: 0.4842 - acc: 0.8167 - val_loss: 1.8422 - val_acc: 0.5550
Epoch 11/35
40s - loss: 0.4158 - acc: 0.8353 - val_loss: 1.5994 - val_acc: 0.5475
Epoch 12/35
41s - loss: 

KeyboardInterrupt: 

### Demo

In [21]:
vgg = Vgg16BN(size=(270, 480), n_classes=nb_classes, lr=lr, batch_size=batch_size, dropout=dropout)
vgg.build()

model = vgg.model
model.load_weights(saved_model_path + '0.03-loss_2epoch_480x270_0.3-dropout_0.001-lr_vggbn.h5')

ValueError: Dimension 0 in both shapes must be equal, but are 61440 and 25088 for 'Assign_215' (op: 'Assign') with input shapes: [61440,4096], [25088,4096].

In [14]:
valid_path

'/home/ipl/Downloads/0126/train/nc-fish-classification/nc_fish_classification/data/fish_detection/valid/'

In [15]:
sample_fish_fns = [fn for fn in glob(valid_path + '*/*.jpg') if "NoF" not in fn]
sample_nof_fns = [fn for fn in glob(valid_path + '*/*.jpg') if "NoF" in fn]

print("fish:", len(sample_fish_fns))
print("no fish:", len(sample_nof_fns))

fish: 435
no fish: 65


In [17]:
# Fish examples

ix = random.randint(0, len(sample_fish_fns))

im = imresize(load_img(sample_fish_fns[ix]), size=(270, 480))
test_x = img_to_array(im).reshape(1, 3, 270, 480)

pred = model.predict(test_x)[0]

print("Ground Truth: Fish\n")
print("Fish Prob:", pred[0])
print("NoFish Prob:", pred[1])

plt.imshow(im)

NameError: name 'model' is not defined

In [ ]:
# NoFish examples

ix = random.randint(0, len(sample_nof_fns))

im = imresize(load_img(sample_nof_fns[ix]), size=(270, 480))
test_x = img_to_array(im).reshape(1, 3, 270, 480)

pred = model.predict(test_x)[0]

print("Ground Truth: NoFish\n")
print("Fish Prob:", pred[0])
print("NoFish Prob:", pred[1])

plt.imshow(im)